In [ ]:
"""
Created on Fri Apr 8 10:29 2022

Prepare reference metrics in NEMO for tuning and evaluation

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf
from multimelt.constants import *
import matplotlib as mpl
import glob

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
nemo_run = 'bi646' # 'bf663','bi646' 

READ IN DATA

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_plumes='/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'
inputpath_boxes='/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
outputpath = '/bettik/burgardc/DATA/NN_PARAM/interim/'

In [ ]:
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)#.chunk(chunks={'x': chunk_size, 'y': chunk_size})

In [ ]:
ds_melt = xr.open_dataset(inputpath_data+'cavity_melt_allyy_Ant_stereo.nc')#, chunks={'x': chunksize, 'y': chunksize})
ds_melt = ds_melt.assign_coords({'time': file_conc.time})
ds_melt_cut = uf.cut_domain_stereo(ds_melt, map_lim, map_lim)


INTEGRATED MELT

In [ ]:
for tt in tqdm(range(1970,2051)):
    
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    file_conc_yy = file_conc_cut['isfdraft_conc'].sel(time=tt)
    ds_melt_yy = ds_melt_cut.sel(time=tt)
    
    ### 2D
    isf_mask = (file_isf['ISF_mask'] == file_isf.Nisf)
    
    xx = file_isf.x
    yy = file_isf.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values
    grid_cell_area = abs(dx*dy)
    
    melt_m_ice_per_y_whole_cell = -1*(ds_melt_yy['melt_cavity']*yearinsec/rho_i) # convert from kg/m2/s
    melt_m_ice_per_y = melt_m_ice_per_y_whole_cell/file_conc_yy # correct by ice shelf concentration
    melt_m_ice_per_y.to_dataset(name='melt_m_ice_per_y').to_netcdf(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')
    
    ### 1D
    chunked_mask = isf_mask.chunk({'x': 500, 'y': 500})
    file_isf_conc = file_conc_yy.where(chunked_mask)
    melt_m_ice_per_y_avg = melt_m_ice_per_y_whole_cell.where(chunked_mask).mean(['x','y'])
    
    NEMO_melt_rates_1D = xr.Dataset({'melt_m_ice_per_y_avg': (melt_m_ice_per_y_avg.dims, melt_m_ice_per_y_avg.data)}, 
                                coords=melt_m_ice_per_y_avg.coords)
    
    NEMO_melt_rates_1D['melt_Gt_per_y_tot'] = -1*ds_melt_yy['melt_cavity'].where(chunked_mask).sum(['x','y']) * grid_cell_area * yearinsec / 10**12

    NEMO_melt_rates_1D['isf_area'] = (isf_mask*file_isf_conc).sum(['x','y'])*grid_cell_area
    NEMO_melt_rates_1D.to_netcdf(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS_'+str(tt)+'.nc','w')

MELT NEAR THE GROUNDING LINE

In [ ]:
### BOX 1
for tt in tqdm(range(1970,1970 + 72)):
    
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    
    file_conc_yy = file_conc_cut['isfdraft_conc'].sel(time=tt)
    
    xx = file_isf.x
    yy = file_isf.y
    dx = (xx[2] - xx[1]).values
    dy = (yy[2] - yy[1]).values

    grid_cell_area = abs(dx*dy)  
    grid_cell_area_weighted = file_conc_yy * grid_cell_area

    isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord').drop('time')

    # Box characteristics
    box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(tt)+'_merged75.nc')
    box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(tt)+'_merged75.nc')
    box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))

    box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=1).drop('Nisf')

    # Reference 2D 
    ref_melt_2D = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_'+str(tt)+'.nc')

    # Reference (in m/yr)
    box1_list = []
    ref_box1_myr_list = []

    for kisf in file_isf.Nisf.values:
        isf_conc_isf = uf.choose_isf(file_conc_yy, isf_stack_mask, kisf)
        box_loc_config_stacked = uf.choose_isf(box1, isf_stack_mask, kisf)
        ref_melt_2D_box1_isf = uf.choose_isf(ref_melt_2D['melt_m_ice_per_y'], isf_stack_mask, kisf).where(np.isfinite(box_loc_config_stacked))
        ref_melt_2D_box1_isf_mean = uf.weighted_mean(ref_melt_2D_box1_isf,['mask_coord'], isf_conc_isf)
        ref_box1_myr_list.append(ref_melt_2D_box1_isf_mean)

    ref_box1_myr_all = xr.concat(ref_box1_myr_list, dim='Nisf')
    ref_box1_myr_all.to_dataset(name='mean_melt_box1_myr').to_netcdf(outputpath_melt+'melt_rates_box1_NEMO_oneFRIS_'+str(tt)+'.nc')